In [1]:
%%capture
!pip install mne

In [2]:
from glob import glob
import scipy.io
import numpy as np
import mne

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Accessing the dataset
IDD='/content/drive/MyDrive/Rest'
TDC='/content/drive/MyDrive/Rest (1)'

In [5]:
#Converting the matlab file to mne
def convertmat2mne(data):
    ch_names = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]
    ch_types = ["eeg"] * 14
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=128)
    info.set_montage("standard_1020")
    data=mne.io.RawArray(data, info)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
    return epochs.get_data()

In [6]:
%%capture
#Importing and converting IDD files to mne
idd_subject=[]
for idd in glob(IDD+'/*.mat'):
    data=scipy.io.loadmat(idd)['clean_data']
    data=convertmat2mne(data)
    idd_subject.append(data)

In [7]:
%%capture
#Importing and converting TDC files to mne
tdc_subject=[]
for tdc in glob(TDC+'/*.mat'):
    data=scipy.io.loadmat(tdc)['clean_data']
    data=convertmat2mne(data)
    tdc_subject.append(data)

In [8]:
len(idd_subject),len(tdc_subject)

(7, 7)

In [9]:
#Creating labels for the files
control_epochs_labels=[len(i)*[0] for i in tdc_subject] #label all epochs in each control file zero
patient_epochs_labels=[len(i)*[1] for i in idd_subject] #label all epochs in each patient file one
len(control_epochs_labels), len(patient_epochs_labels)

(7, 7)

In [10]:
#We need to combine both data and the labels, to form a dataset for the NN
data_list=tdc_subject+idd_subject #one dataset from TDC and IDD data
label_list=control_epochs_labels+patient_epochs_labels #one dataset from TDC and IDD labels
len(data_list), len(label_list)

(14, 14)

In [11]:
#Grouping the data - Avoid grouping based on epochs or the machine learning model would have seen the unseen data, group based on participants
groups=[[i]*len(j) for i,j in enumerate(data_list)]
len(groups)

14

In [12]:
#Create 5 Fold cross-validation loop and train the model for each loop
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler

#Implementing a scaler for a 3D matrix
class StandardScaler3D(BaseEstimator, TransformerMixin):
    # batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        # Reshape to (batch*sequence, channels) and fit the scaler
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self, X):
        # Reshape to (batch*sequence, channels), transform, and reshape back
        X_scaled = self.scaler.transform(X.reshape(-1, X.shape[2]))
        return X_scaled.reshape(X.shape)


In [13]:
#Reshape data because CNN expects data in a particular shape. First check shape by converting list to array
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups)
data_array=np.moveaxis(data_array,1,2)

data_array.shape, label_array.shape, group_array.shape

((420, 512, 14), (420,), (420,))

In [34]:
#Splitting the train and val data for each group
accuracy=[]
for train_index,val_index in gkf.split(data_array,label_array,groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.fit_transform(val_features)

    break

In [35]:
train_features.shape, val_features.shape

((330, 512, 14), (90, 512, 14))

#Keras Implementation

In [37]:
#Import the libraries for keras
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model

In [38]:
#Implementing the first inception module
def block(input):
  conv1=Conv1D(filters=32,kernel_size=2,strides=2,activation='relu',padding='causal')(input)
  conv2=Conv1D(filters=32,kernel_size=4,strides=2,activation='relu',padding='causal')(input)
  conv3=Conv1D(filters=32,kernel_size=8,strides=2,activation='relu',padding='causal')(input)
  x=concatenate([conv1,conv2,conv3], axis=2)
  return x

In [39]:
#Specifying the input shape
input=Input(shape=(512,14))
block1=block(input)
block1.shape

TensorShape([None, 256, 96])

In [40]:
#Implementing the remaining inception layers
block2=block(block1)
block3=block(block2)
block3.shape

TensorShape([None, 64, 96])

In [41]:
#Implementing the GRU layers - RNN for 1D data
gru1=GRU(units=32,return_sequences=True)(block3)
gru2=GRU(units=32,return_sequences=True)(gru1)
gru_out=concatenate([gru1,gru2],axis=2)
gru3=GRU(units=32,return_sequences=True)(gru_out)
gru_out=concatenate([gru1,gru2,gru3],axis=2)
gru_out.shape

TensorShape([None, 64, 96])

In [42]:
#Final GRU layer
gru4=GRU(units=32)(gru_out)
out=Dense(1,activation='sigmoid')(gru4)

In [43]:
#Initailizing the model
model=Model(inputs=input,outputs=out)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 512, 14)]            0         []                            
                                                                                                  
 conv1d_27 (Conv1D)          (None, 256, 32)              928       ['input_4[0][0]']             
                                                                                                  
 conv1d_28 (Conv1D)          (None, 256, 32)              1824      ['input_4[0][0]']             
                                                                                                  
 conv1d_29 (Conv1D)          (None, 256, 32)              3616      ['input_4[0][0]']             
                                                                                            

In [44]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [47]:
model.fit(train_features,train_labels,validation_data=(val_features,val_labels),epochs=20, batch_size=128)

Epoch 1/20
3/3 [==============================] - 2s 767ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.1803 - val_accuracy: 0.9333
Epoch 2/20
3/3 [==============================] - 2s 763ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.3484 - val_accuracy: 0.8889
Epoch 3/20
3/3 [==============================] - 3s 924ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.3978 - val_accuracy: 0.8889
Epoch 4/20
3/3 [==============================] - 3s 962ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.3357 - val_accuracy: 0.9000
Epoch 5/20
3/3 [==============================] - 2s 645ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.2515 - val_accuracy: 0.9333
Epoch 6/20
3/3 [==============================] - 2s 730ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1937 - val_accuracy: 0.9556
Epoch 7/20
3/3 [==============================] - 2s 458ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1660 - val_accuracy: 0.9556
Epoch 8/20
3/3 [====

In [48]:
model.evaluate(val_features,val_labels)

3/3 [==============================] - 0s 49ms/step - loss: 0.0946 - accuracy: 0.9778


[0.09462137520313263, 0.9777777791023254]

In [49]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
